# Exercise 1.04: Tic-Tac-Toe Static Evaluation with a Heuristic Function
In this exercise, you will be performing a static evaluation on the tic-tac-toe game using a heuristic function.

1.- Reuse the code from Steps 1–5 of Activity 1.01

In [1]:
from random import choice

combos_index = [ 
    [0, 1, 2],
    [0, 3, 6],
    [0, 4, 8],
    [1, 4, 7],
    [2, 4, 6],
    [2, 5, 8],
    [3, 4, 5],
    [6, 7, 8]
]

vacio = '?'
IA = 'O'
yo = 'X'


def imprimir(tablero):
    print(" ")
    print(' '.join(tablero[:3]))
    print(' '.join(tablero[3:6]))
    print(' '.join(tablero[6:]))
    print(" ")
    
    
def moverJugador(tablero, fila, columna):
    index = 3 * (fila - 1) + (columna - 1)
    if tablero[index] == vacio:
        return tablero[:index] + vacio + tablero[index + 1:]

    return tablero


def movimientos(tablero, sign):
    boards = list()
    
    for i, v in enumerate(tablero):
        if v == vacio:
            boards.append(tablero[:i] + sign + tablero[i+1:])
    
    return boards


def IAMov(tablero):
    return choice(movimientos(tablero, IA))


def ganarJuego(tablero):
    for combo_index in combos_index:
        if tablero[combo_index[0]] == tablero[combo_index[1]] == tablero[combo_index[2]] != vacio:
            return tablero[combo_index[0]]

    return vacio


def game_loop():
    tablero = vacio * 9
    juegoTerminado = False

    while tablero.count(vacio) != 0 and not juegoTerminado:
        if tablero.count(vacio) % 2 == 0:
            tablero = IAMov(tablero)
        else:
            row = int(input('Fila: '))
            col = int(input('Columna: '))
            tablero = moverJugador(tablero, row, col)

        imprimir(tablero)
        sign_flag = ganarJuego(tablero)

        if sign_flag != vacio:
            juegoTerminado = True

            if sign_flag == IA:
                print("IA ganadora")
            else:
                print("Jugador ganador")
                

def movimient(boards, sign):
    listaMov = list()

    for tablero in boards:
        listaMov.extend(movimientos(tablero, sign))

    return listaMov


def filter_wins(boards, tablaIA, tablaYo):
    for tablero in boards:
        if ganarJuego(tablero) == IA:
            tablaIA.append(tablero)
            boards.remove(tablero)
        
        elif ganarJuego(tablero) == yo:
            tablaYo.append(tablero)
            boards.remove(tablero)

    return boards, tablaIA, tablaYo


def count_possibilities():
    boards = [vacio * 9]
    tablaIA = list()
    tablaYo = list()

    for i in range(9):
        print(f'Step: ' + str(i) + '. Moves: ' + str(len(boards)))
        sign = IA if i % 2 == 0 else yo
        
        boards = movimient(boards, sign)
        filter_wins(boards, tablaIA, tablaYo)
        
    print('First player wins: ' + str(len(tablaIA)))
    print('Second player wins: ' + str(len(tablaYo)))
    print('Draw', str(len(boards)))
    print('Total', str(len(tablaIA) + len(tablaYo) + len(boards)))
    

2.- Create a function that takes the board as input and returns $0$ if the cell is empty, and $-1$ if it's not empty

In [2]:
def iniciarUtilidad(tablero):
    return [0 if cell == vacio else -1 for cell in tablero]

3.- Create a function that takes the utility vector of possible moves, takes three indices inside the utility vector representing a triple, and returns a function.

  > **Hints**  
  > the returned function will expect a `points` parameter and the `utilities` vector as input and will add points to each cell in $(i, j, k)$, as long as the original value of that cell is non-negative $(>=0)$. In other words, we increased the utility of empty cells only.

In [3]:
def generate_add_score(utilities, i, j, k):
    def add_score(points):
        if utilities[i] >= 0:
            utilities[i] += points
        if utilities[j] >= 0:
            utilities[j] += points
        if utilities[k] >= 0:
            utilities[k] += points
    return add_score


4.- Now, create the utility matrix belonging to any board constellation where you will add the `generate_add_score` function defined previously to update the score. You will also implement the rules that we discussed prior to this activity. These rules are as follows:
  * Two AI signs in a row, column, or diagonal, and the third cell is empty: +1000 for the empty cell.
  * The opponent has two signs in a row, column, or diagonal, and the third cell is empty: +100 for the empty cell.
  * One AI sign in a row, column, or diagonal, and the other two cells are empty: +10 for the empty cells.
  * No AI or opponent signs in a row, column, or diagonal: +1 for the empty cells.

In [4]:
def matrizUtilidad(tablero):
    utilities = iniciarUtilidad(tablero)
    
    for [i, j, k] in combos_index:
        add_score = generate_add_score(utilities, i, j, k)
        triple = [tablero[i], tablero[j], tablero[k]]
    
        if triple.count(vacio) == 1:
        
            if triple.count(IA) == 2:
                add_score(1000)
            
            elif triple.count(yo) == 2:
                add_score(100)
        
        elif triple.count(vacio) == 2 and triple.count(IA) == 1:
            add_score(10)
        
        elif triple.count(vacio) == 3:
            add_score(1)
            
    return utilities

5.- Create a function that selects the move with the highest utility value. If multiple moves have the same utility, the function returns both moves

In [5]:
def mejoresMovs(tablero, sign):
    boards = []
    utilities = matrizUtilidad(tablero)
    maUtil = max(utilities)
    
    for i, v in enumerate(tablero):
        if utilities[i] == maUtil:
            boards.append(tablero[:i] + sign + tablero[i+1:])
    
    return boards

def movimient(boards, sign):
    listaMov = list()
    conseguirMov = mejoresMovs if sign == IA else movimientos
    
    for tablero in boards:
        listaMov.extend(conseguirMov(tablero, sign))
        
    return listaMov

6.- Run the application.

Output:

```
step 0. Moves: 1
step 1. Moves: 1
step 2. Moves: 8
step 3. Moves: 24
step 4. Moves: 144
step 5. Moves: 83
step 6. Moves: 214
step 7. Moves: 148
step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607
```

In [6]:
count_possibilities()

Step: 0. Moves: 1
Step: 1. Moves: 1
Step: 2. Moves: 8
Step: 3. Moves: 24
Step: 4. Moves: 144
Step: 5. Moves: 83
Step: 6. Moves: 214
Step: 7. Moves: 148
Step: 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607


By completing this exercise, we have observed that the AI is underperforming compared to our previous activity, Activity 1.03, Fixing the First and Second Moves of the AI to Make It Invincible. In this situation, hardcoding the first two moves was better than setting up the heuristic, but this is because we haven't set up the heuristic properly.